In [17]:
import py2neo

airport_names = ["Dublin", "Paris", "London", "Malaga"]

print "CREATE", ",".join ( [ '({name}:Airport {{name:"{name}"}})'.format(name=a) for a in airport_names])

CREATE (Dublin:Airport {name:"Dublin"}),(Paris:Airport {name:"Paris"}),(London:Airport {name:"London"}),(Malaga:Airport {name:"Malaga"})


In [18]:
from py2neo import authenticate, Graph, Relationship
g = Graph("http://localhost:7474/db/data/")
g.delete_all()

In [19]:
from py2neo import Node
try:
    g.schema.create_uniqueness_constraint("Airport", "name")
except:
    pass

nodes = [g.create(Node("Airport", name=a)) for a in airport_names]

In [20]:
airports = list(g.find("Airport"))

In [21]:
import itertools, random
potential_flights = list ( itertools.permutations ( airports, 2) )
r = random.Random(117)

for i in range(16):
    to, dest = r.choice(potential_flights)
    print to, dest
    g.create(Relationship(to, "Flies_To", dest, cost = random.randint(20,100)))

(dublin:Airport {name:"Dublin"}) (malaga:Airport {name:"Malaga"})
(dublin:Airport {name:"Dublin"}) (london:Airport {name:"London"})
(paris:Airport {name:"Paris"}) (london:Airport {name:"London"})
(dublin:Airport {name:"Dublin"}) (london:Airport {name:"London"})
(paris:Airport {name:"Paris"}) (london:Airport {name:"London"})
(dublin:Airport {name:"Dublin"}) (london:Airport {name:"London"})
(london:Airport {name:"London"}) (dublin:Airport {name:"Dublin"})
(london:Airport {name:"London"}) (dublin:Airport {name:"Dublin"})
(paris:Airport {name:"Paris"}) (london:Airport {name:"London"})
(malaga:Airport {name:"Malaga"}) (dublin:Airport {name:"Dublin"})
(dublin:Airport {name:"Dublin"}) (paris:Airport {name:"Paris"})
(malaga:Airport {name:"Malaga"}) (london:Airport {name:"London"})
(paris:Airport {name:"Paris"}) (malaga:Airport {name:"Malaga"})
(dublin:Airport {name:"Dublin"}) (london:Airport {name:"London"})
(paris:Airport {name:"Paris"}) (london:Airport {name:"London"})
(paris:Airport {name:"

In [22]:
# From previous notebook with new cost
connecting_flights_paris_to_dublin = """
MATCH (paris:Airport) - [f1:Flies_To] -> (an:Airport) - [f2:Flies_To]-> (dublin:Airport)
WHERE paris.name='Paris' AND dublin.name = "Dublin"
RETURN  an, f1.cost+f2.cost AS cost
ORDER BY cost ASC
"""
results = g.run(connecting_flights_paris_to_dublin)
for r in results:
    print r

(u'an': (malaga:Airport {name:"Malaga"}), u'cost': 93)
(u'an': (london:Airport {name:"London"}), u'cost': 193)


In [25]:
# Start in different places
multiple_origins = """
MATCH (paris:Airport) - [f1:Flies_To] -> (an:Airport), (dublin:Airport) - [f2:Flies_To]-> (an:Airport)
WHERE paris.name='Paris' AND dublin.name = "Dublin"
RETURN  *
ORDER BY f1.cost + f2.cost ASC
"""
results = g.run(multiple_origins)

for r in results:
    "NEW FLIGHT/n========="
    print "origins", r['paris'], r['dublin']
    print "destination", r['an'], "\n"

origins (paris:Airport {name:"Paris"}) (dublin:Airport {name:"Dublin"})
destination (malaga:Airport {name:"Malaga"}) 

origins (paris:Airport {name:"Paris"}) (dublin:Airport {name:"Dublin"})
destination (london:Airport {name:"London"}) 

